In [1]:
%matplotlib tk
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
import numpy as np
import scipy.io as sio
import time
#from numba import jit, prange, int32
from numba import jit, prange
import scipy.signal as sg
from scipy.stats import pearsonr
import matplotlib.pyplot as pl
import pygmo as po
import seaborn as sns 
import pandas as pd 
import datetime
from mpl_toolkits.mplot3d import Axes3D
import matplotlib 


In [63]:
class kMcabral():
    """ Kuramoto Model by Cabral without noise"""
    def __init__(self):
        self.tMax     = np.float32(3.0)
        self.tMin     = np.float32(1.0)
        self.fs       = np.float32(250.0)
        self.omega    = np.float32(2*np.pi* 40.0)
        self.dlt      = np.float32(1.0)
        self.pathData = '/home/oscar/Documents/MATLAB/Kuramoto/Cabral/'
        self.nameDTI  = 'AAL_matrices.mat'         # anatomical network
        self.nameFC   = 'Real_Band_FC.mat'     # Empirical Functional connectivity
        self.dt       = np.float32(2e-4)
        self._getEmpiricalData()
        self._desingFilterBands()
        self.log      = np.empty((0,5))
        
    def get_mylogs(self):
        return self.log
    
    def get_name(self):
        return "KM Cabral"
          
    def _getEmpiricalData(self): 
        # load anatomical data
        loaddata    = self.pathData + self.nameDTI
        dti         = sio.loadmat(loaddata)     # C: structural connectivity, D: distance between areas.
        self.D      = dti['D']                  # Distances beween nodes
        anato       = dti['C']                  # Strucural/anatomical network
        self.C      = np.shape(self.D)[1]       # number of nodes/brain areas
        self.anato  = anato / np.mean(anato[~np.identity(self.C,dtype=bool)])   # normalize structural network to a mean = 1
        # load functional data
        loaddata    = self.pathData + self.nameFC
        empiri      = sio.loadmat(loaddata)              # fBands: frequency bands, FCf:functional connectivity
        self.fBands = empiri['freq_bands'].astype(float) # bandpass filter frequency bands
        self.empiFC      = empiri['FC_Env_mean']              # empiprical functional connectivity
        self.empiProfile = []
        for ix in range(0,self.fBands.shape[0]):         # Profile Empirical data
            empi1            = self.empiFC[ix,...]
            self.empiProfile = np.append(self.empiProfile, empi1[np.triu_indices(self.C,1)])
        self.empiProfile = np.clip(self.empiProfile, a_min=0, a_max=None)
        
    def _desingFilterBands(self):
        nyq   = self.fs / 2.0
        trans = 2.0
        self.coeFil = []
        for freq in self.fBands:
            # Filter frequency bands
            passCut = freq / nyq
            stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]
            self.coeFil.append(sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos'))
        # Filter envelops
        self.coeFilEnv = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos')
    
    def _doKuramotoOrder(self, z):
        # global order
        ordG        = np.abs( np.mean( z[:,int(self.tMin*self.fs):], axis = 0 ))
        orderG      = np.mean(ordG)
        orderGstd   = np.std(ordG)
        return orderG, orderGstd,    
    
    def fitness(self,x):
        vel     = x[0]
        kG      = x[1]
        kS      = self.getAnatoCoupling(kG)
        dlayStep, maxDlay = self.getDelays(vel)
        phi     = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay(phi,dlayStep)
        z    = kMcabral._KMcabral(phi,maxDlay,dlayStep,self.fs,self.dt,kS,self.omega)
        self.z = z
        fit, self.simuProfile = self._fitFilterBands(z)
        orderG, orderGstd = self._doKuramotoOrder(z)
        self.log    = np.vstack((self.log, np.array([fit,vel,kG,orderG,orderGstd])))
        return np.array([fit])
    
    def doIndexDelay(self,r,dlayStep):
        commuOff = np.arange(0,r.shape[0]) * r.shape[1]
        commuOff = np.tile(commuOff,(r.shape[0],1)).T
        outpu = dlayStep + commuOff
        return outpu
    
    def getAnatoCoupling(self,kG):
        """Get anatomical network with couplings"""
        kS = self.anato * kG / self.C        # Globa  coupling
        np.fill_diagonal(kS, 0)              # Local coupling
        return np.float32(kS)
    
    def getDelays(self,vel):
        """Return maximum delay and delay steps in samples"""
        dlay        = self.D / (1000.0 * vel)                # [seconds] correct from mm to m
        dlayStep    = np.around(dlay / self.dt).astype(np.int64)  # delay on steps backwards to be done
        maxDlay     = np.int64(np.max(dlayStep))                  # number of time steps for the longest delay
        return dlayStep, maxDlay
    
    def get_bounds(self):
        """Boundaries on: velocity, Kl, Kg"""
        return ([0.1, 0.01],[25, 5000])
    
    def _fitFilterBands(self,z):
        simuProfile = []
        for coefsos in self.coeFil:
            # filter frequency bands
            zFilt   = sg.sosfiltfilt(coefsos, np.imag(z), axis=1, padtype='odd')
            zEnv    = np.abs(sg.hilbert(zFilt, axis=1))
            # filter envelope
            zEnvFilt= sg.sosfiltfilt(self.coeFilEnv, zEnv, axis=1, padtype='odd')
            # Correlation discarding warmup time
            envCo = np.corrcoef(zEnvFilt[:,int(self.tMin*self.fs):-int(self.tMin*self.fs/2)], rowvar=True)
            # set to zero negative correlations
            envCo = np.clip(envCo, a_min=0, a_max=None)
            simuProfile  = np.append(simuProfile, envCo[np.triu_indices(z.shape[0],1)])
        #print(simuProfile.shape)
        ccoef, pval = pearsonr(simuProfile, self.empiProfile)
        return -1 * ccoef, simuProfile
    
    #complex128[:,:](float32[:,:],float32[:,:],int64,int64[:,:],float32,float32,float32,float32,float32[:,:],float32), 
    @jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMcabral(phi,maxDlay,dlayStep,fs,dt,kS,omga):
        C       = phi.shape[0]
        #nodes   = range(C)        
        pi2     = np.float32(2 * np.pi)
        phidt   = np.zeros(C, dtype=np.float32)
        difKsin = np.zeros((C,C), dtype=np.float32)
        phiDif  = np.zeros((C,C), dtype=np.float32)
        for n in range(maxDlay, phi.shape[1]-1):
            idD     = n - dlayStep
            phi_r = phi.ravel()
            for s in prange(C):
                phiDif[:,s]       = phi_r[idD[:,s]] - phi[s,n]            
            difKsin = kS * np.sin(phiDif)
            phidt    = np.sum(difKsin, axis=0) 
            phi_n = phi[:,n] + dt * (phidt+omga)
            phi[:,n+1]  = np.remainder(phi_n, pi2)
            
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        phi = phi[:,::np.int64(1./(fs*dt))]
        return 1.0 * np.exp(1j* phi)
    
    def _doNodeContainers(self,maxDlay):      
        # node's variables
        #import pdb; pdb.set_trace()
        phi         = np.empty((self.C, int(self.tMax/self.dt + maxDlay)), dtype=np.float32) # node phase parameter [C, Nsamples to integrate]
        # initial conditions as history for the time delays
        omegaT      = self.omega * np.linspace(0, maxDlay*self.dt+self.dt,maxDlay+1, dtype=np.float32)
        phi[:,0:maxDlay+1]  = np.tile(np.remainder(omegaT,2*np.pi),(self.C,1))
        return phi
    
    def setVariablesModel(self,x):
        "set the basic variable need to run the KM model"
        vel     = x[0]
        kG      = x[1]
        self.kS      = self.getAnatoCoupling(kG)
        self.dlayStep, self.maxDlay = self.getDelays(vel)
        self.phi     = self._doNodeContainers(self.maxDlay)
        self.dlayId  = self.doIndexDelay(self.phi,self.dlayStep)
        
        
        

/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [64]:
kmc = kMcabral()
kmc.setVariablesModel([0.9,150])
#outKM =  kMcabral._KMcabral(kmc.phi,kmc.maxDlay,kmc.dlayStep,kmc.fs,kmc.dt,kmc.kS,kmc.omega)
#kmc.fitness([0.9,150])
#%timeit kmc.fitness([0.9,150])

In [65]:
%timeit  kMcabral._KMcabral(kmc.phi,kmc.maxDlay,kmc.dlayStep,kmc.fs,kmc.dt,kmc.kS,kmc.omega)

1.28 s ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit  kMcabral._KMcabral(kmc.phi,kmc.maxDlay,kmc.dlayStep,kmc.fs,kmc.dt,kmc.kS,kmc.omega)

502 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
0.34 * 500 * 20 / 3600

0.9444444444444444

# Run Evolution

In [68]:
generations = 500
sizePop     = 25
# algorithm
algo   = po.algorithm(po.de1220(gen=generations,
                                allowed_variants=np.arange(1,19),
                                variant_adptv=1,
                                ftol=1e-4,
                                xtol=1e-4))


algo.set_verbosity(1)
# problem
prob   = po.problem(kMcabral())
# population
pop    = po.population(prob=prob, size=sizePop)
# evolution
popE    = algo.evolve(pop)
# Results
loguda = algo.extract(po.de1220).get_log()
# get best fitness per generation
best = np.array([loguda[i][2] for i in range(len(loguda))])
# get parameter for the logging variable in problem class
probE    = popE.problem.extract(kMcabral)
logged   = probE.get_mylogs()
fitness  = logged[:,0]
velocity = logged[:,1]
kG       = logged[:,2]
orderG   = logged[:,2]
orderGsd = logged[:,2]
#save file

AttributeError: 'kMcabral' object has no attribute 'get_logs'

In [73]:
%store kG
%store velocity
%store fitness

Stored 'kG' (ndarray)
Stored 'velocity' (ndarray)
Stored 'fitness' (ndarray)


# Profiling memory usage

In [44]:
%%file test3Memot.py
import numpy as np

def KMcabral(phi,maxDlay,dlayStep,fs,dt,kS,omga):
    C       = phi.shape[0]
    #nodes   = range(C)        
    pi2     = np.float32(2 * np.pi)
    phidt   = np.zeros(C, dtype=np.float32)
    difKsin = np.zeros((C,C), dtype=np.float32)
    phiDif  = np.zeros((C,C), dtype=np.float32)
    for n in range(maxDlay, phi.shape[1]-1):
        idD     = n - dlayStep
        phi_r = phi.ravel()
        for s in range(C):
            #phiDif       = phi[nodes, idD[:,s]] - phi[s, n]
            #phiDif       = phi_r[idD[:,s]] - phi[s,n]
            phiDif[:,s]       = phi_r[idD[:,s]] - phi[s,n]

            #difKsin[:,s] = kS[:,s] * np.sin(phiDif)

        difKsin = kS * np.sin(phiDif)

        phidt    = np.sum(difKsin, axis=0) 
        phi_n = phi[:,n] + dt * (phidt+omga)
        phi[:,n+1]  = np.remainder(phi_n, pi2)

    phi     = phi[:,maxDlay+1:]
    # simple downsampling (there may be aliasing)
    phi = phi[:,::np.int64(1./(fs*dt))]
    return 1.0 * np.exp(1j* phi)

Writing test3Memot.py


In [56]:
from test3Memot import KMcabral
import numpy as np
%mprun -f KMcabral KMcabral(kmc.phi,kmc.maxDlay,kmc.dlayStep,kmc.fs,kmc.dt,kmc.kS,kmc.omega)

In [55]:
from test3Memot import KMcabral

%memit KMcabral(kmc.phi,kmc.maxDlay,kmc.dlayStep,kmc.fs,kmc.dt,kmc.kS,kmc.omega)

peak memory: 256.03 MiB, increment: 0.57 MiB
